## 라이브러리 불러오기

In [ ]:
!pip install -q peft transformers datasets
!pip install bitsandbytes
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00


In [ ]:
from transformers import  AutoConfig, BitsAndBytesConfig
# from transformers import LlamaForCausalLM, LlamaTokenizerFast , default_data_collator, get_linear_schedule_with_warmup
# from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast , default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm




In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
text_column = "message"
label_column = "keyword"
max_length = 64


## 데이터 셋 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/dataset"

/content/drive/MyDrive/dataset


In [ ]:
data_files = {"train": "./output_train.json", "test": "./output_test.json"}

dataset = load_dataset('json', data_files=data_files )
dataset['train'][0]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'message': '굳 완전 편함 ㅋㅋ 평소에도 입고 필라테스 할때도 입는준임 ㅋㅋ 편한거 찾는다면 강추 ㅋㅋ',
 'keyword': '착용감'}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['highlight', 'keyword'],
        num_rows: 13440
    })
    test: Dataset({
        features: ['highlight', 'keyword'],
        num_rows: 3360
    })
})

## 전처리

In [ ]:
classes = set(dataset["train"]["keyword"])



In [ ]:
classes

{'가격',
 '기능성',
 '길이',
 '디자인',
 '라인(핏)',
 '마감처리',
 '배송',
 '사이즈',
 '색상',
 '소재',
 '스타일',
 '신축성',
 '착용감',
 '품질'}

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer = GPTNeoXTokenizerFast.from_pretrained(model_name_or_path)


# tokenizer = LlamaTokenizerFast.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)






# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token_id = tokenizer.eos_token_id
# target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name_or_path,
#     padding=True,
#     truncation=True,
#     max_length=target_max_length,
#     return_tensors = None,
#     return_token_type_ids  = None,
#     )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPTNeoXTokenizerFast'.


4


In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/11188 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/6547 [00:00<?, ? examples/s]

In [ ]:
processed_datasets['train'][0]

{'input_ids': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  80,
  16313,
  18937,
  4361,
  3424,
  5172,
  1130,
  953,
  10609,
  237,
  9664,
  237,
  3426,
  5838,
  745,
  283,
  18679,
  16493,
  823,
  1730,
  309,
  745,
  272,
  731,
  718,
  10609,
  237,
  9664,
  237,
  14816,
  548,
  1044,
  6380,
  631,
  1119,
  10609,
  237,
  9664,
  237,
  1311,
  10144,
  6063,
  4361,
  224,
  1440,
  463,
  748,
  2],
 'attention_mask': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -10

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

## 모델 불러오기

In [ ]:

device = "cuda"
model_name_or_path = "beomi/KoAlpaca-Polyglot-12.8B"
tokenizer_name_or_path = "beomi/KoAlpaca-Polyglot-12.8B"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="리뷰 키워드 분석:",
    tokenizer_name_or_path=model_name_or_path,
)


In [ ]:
lr = 1e-4 #3e-2
num_epochs = 10  #50
batch_size = 8 #8

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
# model = GPTNeoXForCausalLM.from_pretrained(model_name_or_path)
# model = LlamaForCausalLM.from_pretrained(model_name_or_path)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = GPTNeoXForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

trainable params: 40,960 || all params: 12,892,856,320 || trainable%: 0.00031769531113490294
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

## 학습

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 819/819 [03:30<00:00,  3.89it/s]


epoch=0: train_ppl=tensor(5.4965, device='cuda:0') train_epoch_loss=tensor(1.7041, device='cuda:0') eval_ppl=tensor(2.2285, device='cuda:0') eval_epoch_loss=tensor(0.8013, device='cuda:0')


100%|██████████| 819/819 [03:30<00:00,  3.89it/s]


epoch=1: train_ppl=tensor(1.8931, device='cuda:0') train_epoch_loss=tensor(0.6382, device='cuda:0') eval_ppl=tensor(1.9015, device='cuda:0') eval_epoch_loss=tensor(0.6426, device='cuda:0')


100%|██████████| 819/819 [03:30<00:00,  3.88it/s]


...

epoch=7: train_ppl=tensor(1.4081, device='cuda:0') train_epoch_loss=tensor(0.3423, device='cuda:0') eval_ppl=tensor(1.5316, device='cuda:0') eval_epoch_loss=tensor(0.4263, device='cuda:0')


100%|██████████| 819/819 [03:31<00:00,  3.88it/s]


epoch=8: train_ppl=tensor(1.4264, device='cuda:0') train_epoch_loss=tensor(0.3552, device='cuda:0') eval_ppl=tensor(1.5327, device='cuda:0') eval_epoch_loss=tensor(0.4270, device='cuda:0')


100%|██████████| 819/819 [03:30<00:00,  3.88it/s]

epoch=9: train_ppl=tensor(1.3755, device='cuda:0') train_epoch_loss=tensor(0.3188, device='cuda:0') eval_ppl=tensor(1.5073, device='cuda:0') eval_epoch_loss=tensor(0.4103, device='cuda:0')


In [ ]:
peft_model_id = "dongseon/cllama3"
model.push_to_hub("dongseon/cllama3", use_auth_token=True,create_pr=1)

adapter_model.bin:   0%|          | 0.00/165k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dongseon/cllama3/commit/8e45d09efec86c8fabf676eb0c29c3b3887216b8', commit_message='Upload model', commit_description='', oid='8e45d09efec86c8fabf676eb0c29c3b3887216b8', pr_url='https://huggingface.co/dongseon/cllama3/discussions/2', pr_revision='refs/pr/2', pr_num=2)

## 테스트

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "dongseon/cllama3"

config = PeftConfig.from_pretrained(peft_model_id)
model = GPTNeoXForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
txt = "단추가 너덜 거리고, 화면과 흰색이 보다 누리끼리한 느낌이에요."

In [ ]:
inputs = tokenizer(
    f'{text_column} : {txt} Label : ',
    return_tensors="pt",
)


In [ ]:
inputs

{'input_ids': tensor([[   80, 16313, 18937,  4361,   780, 14970,  1258,  4652,  2137,   283,
            15,  4699,   359, 13508,   270,  2652,  4172,  5049,   288,  2671,
           270,  2733,    17,  1311, 10144,  6063,  4361,   224]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=target_max_length)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:996: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


In [ ]:
decoded_outputs  = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens= False)
filtered_outputs = [output.split('<|endoftext|>')[0].strip() for output in decoded_outputs]

for output in filtered_outputs:
    print(output)



message : 단추가 너덜 거리고, 화면과 흰색이 보다 누리끼리한 느낌이에요. Label : 품질


In [ ]:
outputs

tensor([[   80, 16313, 18937,  4361,   780, 14970,  1258,  4652,  2137,   283,
            15,  4699,   359, 13508,   270,  2652,  4172,  5049,   288,  2671,
           270,  2733,    17,  1311, 10144,  6063,  4361,   224,  7625,    15,
          1177,   398,     2,     6,     6,     6,  9272,    29,  3844,    15,
          7953,    15,  3129,   433,  1392,  1459,   363,  2273]])

In [ ]:
inputs.input_ids

tensor([[   80, 16313, 18937,  4361,   780, 14970,  1258,  4652,  2137,   283,
            15,  4699,   359, 13508,   270,  2652,  4172,  5049,   288,  2671,
           270,  2733,    17,  1311, 10144,  6063,  4361,   224]])

In [ ]:
output_tokens = tokenizer.convert_ids_to_tokens(outputs[0])
print(output_tokens)

['m', 'ess', 'age', 'Ġ:', 'Ġëĭ¨', 'ì¶Ķê°Ģ', 'ĠëĦĪ', 'ëįľ', 'Ġê±°ë¦¬', 'ê³ł', ',', 'ĠíĻĶë©´', 'ê³¼', 'ĠíĿ°ìĥī', 'ìĿ´', 'Ġë³´ëĭ¤', 'ĠëĪĦë¦¬', 'ëģ¼ë¦¬', 'íķľ', 'ĠëĬĲëĤĮ', 'ìĿ´', 'ìĹĲìļĶ', '.', 'ĠL', 'ab', 'el', 'Ġ:', 'Ġ', 'íĴĪì§Ī', ',', 'ìĥī', 'ìĥģ', '<|endoftext|>', '#', '#', '#', 'Ġìŀħëł¥', ':', 'ĠíĴĪì§Ī', ',', 'Ġìĥīìĥģ', ',', 'ĠìĬ¤íĥĢìĿ¼', 'Ġëĵ±', 'Ġëª¨ëĵł', 'Ġë©´', 'ìĹĲìĦľ', 'Ġìļ°ìĪĺ']


In [ ]:
print(tokenizer.convert_tokens_to_string(output_tokens))

message : 단추가 너덜 거리고, 화면과 흰색이 보다 누리끼리한 느낌이에요. Label : 품질,색상<|endoftext|>### 입력: 품질, 색상, 스타일 등 모든 면에서 우수


In [ ]:
# print(tokenizer.vocab_size)
# print(tokenizer.model_max_length)
# print(tokenizer.model_input_names)

In [ ]:
input_ids = tokenizer.encode("품질", add_special_tokens=True, truncation=True, padding=True)
print(input_ids)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[7625]


In [ ]:
decoded_text = tokenizer.decode(input_ids)
print(decoded_text)

품질


In [ ]:
# 두 개의 텍스트
text1 = "품질"
text2 = "너덜 거리고"

# 텍스트를 input_ids로 변환
input_ids1 = tokenizer.encode(text1, add_special_tokens=True, truncation=True, padding=True)
input_ids2 = tokenizer.encode(text2, add_special_tokens=True, truncation=True, padding=True)

# 입력을 PyTorch Tensor로 변환
input_ids1 = torch.tensor(input_ids1).unsqueeze(0)  # 배치 차원 추가
input_ids2 = torch.tensor(input_ids2).unsqueeze(0)  # 배치 차원 추가

# 임베딩 벡터 얻기
with torch.no_grad():
    output1 = model.generate(input_ids=input_ids1, max_length=50,
    num_return_sequences=1, pad_token_id=tokenizer.eos_token_id).float()  # 부동 소수점으로 변환
    output2 = model.generate(input_ids=input_ids2, max_length=50,
    num_return_sequences=1, pad_token_id=tokenizer.eos_token_id).float()  # 부동 소수점으로 변환

# 두 벡터 간의 코사인 유사도 계산
cosine_similarity = torch.nn.functional.cosine_similarity(output1, output2, dim=1)

# 결과 출력
print("코사인 유사도:", cosine_similarity.item())

코사인 유사도: 0.07882975786924362


In [ ]:
# 텍스트를 토큰화하고 input_ids 얻기
input_ids = tokenizer.encode(txt, return_tensors="pt")

# 모델에 입력 전달하여 예측 수행
with torch.no_grad():
    outputs = model(input_ids)

print(outputs)

In [ ]:
# 분류 결과 (예시: 다중 클래스 분류)
logits = outputs.logits
predicted_class_index = torch.argmax(logits, dim=2)[0]  # [0]을 추가하여 첫 번째 샘플에 대한 최대 인덱스를 얻음
predicted_class = tokenizer.decode(predicted_class_index)  # 인덱스를 텍스트로 디코딩

print(logits)
print(predicted_class)

In [ ]:
# 텍스트를 토큰화하고 input_ids 얻기
input_ids = tokenizer.encode(txt, return_tensors="pt")

# 모델 출력에서 특정 토큰의 위치 설정 (예: 세 번째 토큰)
token_position = 2

# 특정 토큰을 예측하도록 구성
input_ids_for_prediction = input_ids.clone()  # 원래 input_ids 복사
input_ids_for_prediction[0, token_position] = tokenizer.pad_token_id  # 예측하려는 토큰을 패딩 토큰으로 대체

# 모델에 예측할 토큰 전달하여 예측 수행
with torch.no_grad():
    outputs = model(input_ids_for_prediction)

# 예측 결과에서 특정 토큰의 로짓 추출
predicted_logits = outputs.logits[0, token_position]

# 손실 정의: 예측된 로짓과 실제 토큰의 로짓 간의 차이 (예: 교차 엔트로피)
loss = torch.nn.functional.cross_entropy(predicted_logits.unsqueeze(0), input_ids[0, token_position].unsqueeze(0))

# 손실을 기반으로 gradient 계산
loss.backward()

# 특정 토큰의 gradient 얻기
token_gradient = input_ids.grad[0, token_position]

# 결과 출력 (특정 토큰의 gradient)
print("특정 토큰의 gradient:", token_gradient)

RuntimeError: ignored